In [29]:
import numpy as np 
import pandas as pd              
import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
import os
import random
from PIL import Image, ImageDraw, ImageFont
from faker import Faker
import qrcode

# --- Configuration ---
output_dir = "aadhaar_kyc_dataset"
num_samples = 1000  # Number of Aadhaar card samples
image_size = (800, 500)  # Aadhaar card size (approx.)
font_path = "arial.ttf"  # Use Noto Sans or a similar Indian font
profile_image_dir = "profile_images"  # Directory to store profile pictures

# --- Create output directories ---
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "labels"), exist_ok=True)
os.makedirs(profile_image_dir, exist_ok=True)  # Profile images folder

# --- Initialize Faker for synthetic data ---
fake = Faker("en_IN")  # Use Indian locale

# --- Helper functions ---
def generate_qr_code(data):
    """Generates a QR code with the given data."""
    qr = qrcode.QRCode(version=1, error_correction=qrcode.constants.ERROR_CORRECT_H, box_size=10, border=4)
    qr.add_data(data)
    qr.make(fit=True)
    qr_img = qr.make_image(fill_color="black", back_color="white").resize((100, 100))  # Resize for Aadhaar
    return qr_img

def generate_profile_image(output_path):
    """Generates or loads a random profile image."""
    # For simplicity, we use placeholders; you can download real images for higher realism
    profile_img = Image.new("RGB", (100, 100), (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)))
    draw = ImageDraw.Draw(profile_img)
    draw.text((20, 40), "Photo", fill="black")  # Placeholder text
    
    profile_img.save(output_path)
    return profile_img

def generate_aadhaar_card():
    """Generates a synthetic Aadhaar card."""
    # Create a blank Aadhaar card template
    aadhaar_card = Image.new("RGB", image_size, "white")
    draw = ImageDraw.Draw(aadhaar_card)

    # Add borders
    draw.rectangle([(10, 10), (790, 490)], outline="black", width=5)

    # Add profile picture
    profile_path = os.path.join(profile_image_dir, f"profile_{random.randint(1, 1000)}.jpg")
    profile_img = generate_profile_image(profile_path)
    aadhaar_card.paste(profile_img, (30, 80))  # Paste photo on Aadhaar card

    # Add QR code
    qr_data = fake.uuid4()
    qr_img = generate_qr_code(qr_data)
    aadhaar_card.paste(qr_img, (670, 80))  # Paste QR code

    # Add Aadhaar details
    name = fake.name()
    dob = fake.date_of_birth().strftime("%d/%m/%Y")
    gender = random.choice(["Male", "Female", "Other"])
    aadhaar_number = "".join([str(random.randint(0, 9)) for _ in range(12)])
    address = fake.address().replace("\n", ", ")

    # Load font
    try:
        font = ImageFont.truetype(font_path, 20)
    except:
        font = ImageFont.load_default()

    # Write text on the card
    draw.text((150, 80), f"Name: {name}", font=font, fill="black")
    draw.text((150, 120), f"DOB: {dob}", font=font, fill="black")
    draw.text((150, 160), f"Gender: {gender}", font=font, fill="black")
    draw.text((150, 200), f"Aadhaar No: {aadhaar_number}", font=font, fill="black")
    draw.text((30, 300), f"Address: {address}", font=font, fill="black")

    # Return generated card and details
    return aadhaar_card, {"name": name, "dob": dob, "gender": gender, "aadhaar_number": aadhaar_number, "address": address}

# --- Main Loop ---
for i in range(num_samples):
    filename = f"aadhaar_{i}"
    
    # Generate Aadhaar card
    aadhaar_card, details = generate_aadhaar_card()
    
    # Save the image
    aadhaar_card.save(os.path.join(output_dir, "images", filename + ".jpg"))
    
    # Save the label
    with open(os.path.join(output_dir, "labels", filename + ".txt"), "w") as f:
        f.write(f"{details}")

print(f"Dataset created with {num_samples} samples in '{output_dir}'")


Dataset created with 1000 samples in 'aadhaar_kyc_dataset'
